In [1]:
from pathlib import Path
from data_import import load_data, load_ecb_1y_yield
from merton_df import build_merton_inputs, equity_volatility

print(Path.cwd())

c:\Users\vkeenan\OneDrive - Delft University of Technology\Documents\University\QRM\Accenture Project\code


In [2]:
# Load Accenture dataset
ret_daily, bs = load_data(
    xlsx_path= None,
    verbose=True
)

print(ret_daily.head())
print(bs.head())

      country_iso          isin       date                       company  \
41651         DEU  DE0005190003 2010-01-05  BAYERISCHE MOTOREN WERKE AKT   
41652         DEU  DE0005190003 2010-01-06  BAYERISCHE MOTOREN WERKE AKT   
41653         DEU  DE0005190003 2010-01-07  BAYERISCHE MOTOREN WERKE AKT   
41654         DEU  DE0005190003 2010-01-08  BAYERISCHE MOTOREN WERKE AKT   
41655         DEU  DE0005190003 2010-01-11  BAYERISCHE MOTOREN WERKE AKT   

        gvkey   shares_out   close  mcap_reported  shares_out_filled  \
41651  100022  601995196.0  32.310   1.945046e+10        601995196.0   
41652  100022  601995196.0  32.810   1.975146e+10        601995196.0   
41653  100022  601995196.0  33.100   1.992604e+10        601995196.0   
41654  100022  601995196.0  32.655   1.965815e+10        601995196.0   
41655  100022  601995196.0  32.170   1.936619e+10        601995196.0   

               mcap  bad_day  logret_close  logret_mcap  
41651  1.945046e+10    False      0.008080     0.008

In [3]:
# Load ECB 1Y risk-free yield data
df_rf = load_ecb_1y_yield(
    startPeriod="2010-01-01",
    endPeriod="2025-12-31",
    out_file="ecb_yc_1y_aaa.xml",
    verify_ssl=False
)

print(df_rf.head())
df_rf.to_csv("ecb_riskfree_1y_daily.csv", index=False)
#rf_1y = pd.read_csv("ecb_riskfree_1y_daily.csv", parse_dates=["date"])


c:\Users\vkeenan\AppData\Local\miniconda3\envs\Accenture\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data-api.ecb.europa.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Data has been written to ecb_yc_1y_aaa.xml
        date      r_1y
0 2010-01-04  0.008330
1 2010-01-05  0.007934
2 2010-01-06  0.007782
3 2010-01-07  0.007491
4 2010-01-08  0.007417


In [ ]:
# Merton model inputs and implementation
merton_inputs = build_merton_inputs(
    ret_daily=ret_daily,
    bs=bs,
    df_rf=df_rf,
    T=1.0,
    equity_col="mcap",
    returns_col="logret_mcap",
    drop_missing_B=True,
    drop_missing_r=True,
)

# Calculate equity volatility
merton_inputs_vol = equity_volatility(
    merton_inputs,
    ret_col="logret_mcap",   # use market-cap log returns (consistent with E=mcap)
    window=252,
    min_obs=126
)

print(merton_inputs_vol[["gvkey","date","E","B","r","sigma_E"]].head())

# Check missingness of sigma_E
print("Missing sigma_E %:", merton_inputs_vol["sigma_E"].isna().mean() * 100)
print(merton_inputs_vol["sigma_E"].describe())

Merton input QA: {'rows_out': 198390, 'n_firms_out': 50, 'date_min_out': '2010-01-05', 'date_max_out': '2025-12-19', 'pct_missing_B_after_merge': 7.1903825797671255, 'pct_missing_r_after_merge': 0.0}
    gvkey       date             E   B         r  sigma_E
0  100022 2010-01-05  1.945046e+10 NaN  0.007934      NaN
1  100022 2010-01-06  1.975146e+10 NaN  0.007782      NaN
2  100022 2010-01-07  1.992604e+10 NaN  0.007491      NaN
3  100022 2010-01-08  1.965815e+10 NaN  0.007417      NaN
4  100022 2010-01-11  1.936619e+10 NaN  0.007056      NaN
